### Advanced in silico drug design workshop. Olomouc, 30 January - 1 February, 2017.
### QSAR tutorial.
Dr. Pavel Polishchuk  
   
Institute of Molecular and Translational Medicine, Faculty of Medicine and Dentistry, Palacký University and University Hospital in Olomouc, Hněvotínská 1333/5, 779 00 Olomouc, Czech Republic  
http://imtm.cz  
http://qsar4u.com  
pavel_polishchuk@ukr.net  
pavlo.polishchuk@upol.cz

In [1]:
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors

In [ ]:
# from rdkit.Chem.Draw import IPythonConsole
# from rdkit.Chem import Draw
# IPythonConsole.ipython_useSVG=True

In [2]:
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, cohen_kappa_score, matthews_corrcoef
# from sklearn.externals import joblib


#### Reading molecules and activity from SDF

In [3]:
fname = "data/logBB.sdf"

mols = []
y = []
for mol in Chem.SDMolSupplier(fname):
    if mol is not None:
        mols.append(mol)
        y.append(mol.GetIntProp("logBB_class"))

In [13]:
len(mols)

321

#### Calculate descriptors (fingerprints) and convert them into numpy array

In [4]:
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]

In [5]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [6]:
x = rdkit_numpy_convert(fp)

In [7]:
x.shape

(321, 2048)

In [15]:
x[1]

array([0., 1., 0., ..., 0., 0., 0.])

In [16]:
# check wether the data set is balanced
sum(y) / len(y)

0.5545171339563862

#### Set random seed to make all further calculations reproducible

In [17]:
seed = 42

#### Split the whole set on training and test sets

In [11]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

In [18]:
print(y_ts)

[0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1]


#### Create folds for cross-validation

In [19]:
cv = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

In [20]:
# print out ids of folds
for i, (train_index, test_index) in enumerate(cv.split(x_tr, y_tr)):
    print("\nFold_" + str(i+1))
    print("TRAIN:", train_index)
    print("TEST:", test_index)


Fold_1
TRAIN: [  0   1   2   3   4   5   7   8  10  13  14  15  16  18  19  21  22  23
  24  25  26  27  28  30  32  36  37  38  39  40  42  43  44  45  46  47
  48  49  52  53  55  56  58  59  61  63  64  65  66  67  68  69  70  71
  72  73  75  76  78  79  80  82  83  84  85  86  87  88  89  90  92  93
  95  96  97  98  99 100 101 103 104 105 106 108 109 112 114 115 116 117
 118 119 120 122 123 124 125 126 127 128 129 130 131 132 134 135 136 137
 138 139 140 142 143 144 145 146 147 148 149 150 153 154 157 158 159 160
 161 162 163 165 166 167 168 169 170 171 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 193 194 195 196 197 198 199
 200 201 203 204 205 206 207 208 209 211 212 213 214 215 216 217 218 219
 220 221 222 223 224 226 229 231 232 233 234 235 236 237 238 241 242 244
 246 247 248 250 251 255]
TEST: [  6   9  11  12  17  20  29  31  33  34  35  41  50  51  54  57  60  62
  74  77  81  91  94 102 107 110 111 113 121 133 141 151 152 155 156 164
 172

#### Scale X

This step may be crucial for certain modeling approaches lke SVM.
In the case of binary fingerprints it may be less useful.

In [21]:
# obtain scale object which can be further applied to scale any data to fit the training set
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [15]:
# it is a good idea to save it for future use
joblib.dump(scale, "data/logBB_scale.pkl", compress=3)

['data/logBB_scale.pkl']

#### Search for optimal tuning parameters and build the model

In [22]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [23]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [24]:
# run model building
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [204, 292, 409, 682],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [21]:
m.best_params_

{'max_features': 682, 'n_estimators': 500}

In [22]:
m.best_score_

0.7930618401206637

In [23]:
m.cv_results_

{'mean_fit_time': array([0.37966509, 0.9311729 , 1.93254223, 0.44379139, 1.07088127,
        2.21075945, 0.48081465, 1.18825803, 2.38702755, 0.59283757,
        1.88748598, 3.04504819]),
 'std_fit_time': array([0.01195491, 0.02994278, 0.03742874, 0.00605149, 0.03140795,
        0.04834872, 0.0092752 , 0.01475086, 0.05169704, 0.03926034,
        0.08532356, 0.18838839]),
 'mean_score_time': array([0.01408706, 0.03168936, 0.06081877, 0.01337733, 0.03363218,
        0.07622437, 0.01290307, 0.03128166, 0.06119289, 0.01764927,
        0.05096202, 0.06072068]),
 'std_score_time': array([0.00137223, 0.0010989 , 0.00123252, 0.00060334, 0.00492082,
        0.02830822, 0.00038682, 0.00159369, 0.00125428, 0.00762632,
        0.02978298, 0.0024502 ]),
 'param_max_features': masked_array(data=[204, 204, 204, 292, 292, 292, 409, 409, 409, 682, 682,
                    682],
              mask=[False, False, False, False, False, False, False, False,
                    False, False, False, False],
  

In [24]:
m.cv_results_['mean_test_score']

array([0.75784314, 0.76176471, 0.78514329, 0.78529412, 0.78914027,
       0.76953243, 0.76953243, 0.7852187 , 0.78129713, 0.78914027,
       0.78137255, 0.79306184])

In [25]:
m.cv_results_['params']

[{'max_features': 204, 'n_estimators': 100},
 {'max_features': 204, 'n_estimators': 250},
 {'max_features': 204, 'n_estimators': 500},
 {'max_features': 292, 'n_estimators': 100},
 {'max_features': 292, 'n_estimators': 250},
 {'max_features': 292, 'n_estimators': 500},
 {'max_features': 409, 'n_estimators': 100},
 {'max_features': 409, 'n_estimators': 250},
 {'max_features': 409, 'n_estimators': 500},
 {'max_features': 682, 'n_estimators': 100},
 {'max_features': 682, 'n_estimators': 250},
 {'max_features': 682, 'n_estimators': 500}]

#### Save model

In [24]:
joblib.dump(m, "data/logBB_rf_morgan.pkl", compress=3)

['data/logBB_rf_morgan.pkl']

#### Predict test set compounds

In [27]:
# load scale if necessary
scale = joblib.load("data/logBB_scale.pkl")

NameError: name 'joblib' is not defined

In [28]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [29]:
# predict logBB class
pred_rf = m.predict(x_ts)

In [30]:
pred_rf

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

#### calc statistics for test set preditions

In [31]:
accuracy_score(y_ts, pred_rf)

0.7538461538461538

In [32]:
matthews_corrcoef(y_ts, pred_rf)

0.4995357946109721

In [33]:
cohen_kappa_score(y_ts, pred_rf)

0.4985535197685632

#### applicability domain estimates

In [34]:
# if the model includes several ones like RF models or consensus models (or for probabilistic models)
# we can calculate consistency of predictions amongs those models and use it for estimation of applicability domain
pred_prob = m.predict_proba(x_ts)

In [35]:
# probablity
pred_prob

array([[0.886, 0.114],
       [0.234, 0.766],
       [0.244, 0.756],
       [0.204, 0.796],
       [0.38 , 0.62 ],
       [0.21 , 0.79 ],
       [0.006, 0.994],
       [0.024, 0.976],
       [0.838, 0.162],
       [0.922, 0.078],
       [0.176, 0.824],
       [0.824, 0.176],
       [0.548, 0.452],
       [0.958, 0.042],
       [0.382, 0.618],
       [0.552, 0.448],
       [0.094, 0.906],
       [0.032, 0.968],
       [0.004, 0.996],
       [0.06 , 0.94 ],
       [0.55 , 0.45 ],
       [0.   , 1.   ],
       [0.224, 0.776],
       [0.546, 0.454],
       [0.238, 0.762],
       [0.368, 0.632],
       [0.612, 0.388],
       [0.31 , 0.69 ],
       [0.832, 0.168],
       [0.628, 0.372],
       [0.024, 0.976],
       [0.966, 0.034],
       [0.12 , 0.88 ],
       [0.81 , 0.19 ],
       [0.292, 0.708],
       [0.58 , 0.42 ],
       [0.926, 0.074],
       [0.792, 0.208],
       [0.654, 0.346],
       [0.   , 1.   ],
       [0.868, 0.132],
       [0.   , 1.   ],
       [0.288, 0.712],
       [0.5

In [36]:
# setup threshold
threshold = 0.8

In [37]:
# calc maximum predicted probability for each row (compound) and compare to the threshold
da = np.amax(pred_prob, axis=1) > threshold

In [38]:
da

array([ True, False, False, False, False, False,  True,  True,  True,
        True,  True,  True, False,  True, False, False,  True,  True,
        True,  True, False,  True, False, False, False, False, False,
       False,  True, False,  True,  True,  True,  True, False, False,
        True, False, False,  True,  True,  True, False, False, False,
        True,  True, False,  True, False, False,  True, False, False,
        True,  True, False,  True,  True,  True,  True, False,  True,
       False,  True])

In [39]:
# calc statistics
accuracy_score(np.asarray(y_ts)[da], pred_rf[da])

0.8529411764705882

In [40]:
matthews_corrcoef(np.asarray(y_ts)[da], pred_rf[da])

0.7050707456243541

In [41]:
cohen_kappa_score(np.asarray(y_ts)[da], pred_rf[da])

0.7038327526132404

In [42]:
# calc coverage
sum(da) / len(da)

0.5230769230769231

#### Build SVM model

In [43]:
# create grid search dictionary
param_grid = {"C": [10 ** i for i in range(0, 5)],
              "gamma": [10 ** i for i in range(-6, 0)]}

In [44]:
# setup model building
svm = GridSearchCV(SVC(kernel='rbf', probability=True), param_grid, n_jobs=2, cv=cv, verbose=1)

In [45]:
# run model building
svm.fit(x_tr, y_tr)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=SVC(probability=True), n_jobs=2,
             param_grid={'C': [1, 10, 100, 1000, 10000],
                         'gamma': [1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1]},
             verbose=1)

In [46]:
svm.best_score_

0.7540723981900452

In [47]:
svm.best_params_

{'C': 100, 'gamma': 1e-05}

In [46]:
# save model
joblib.dump(svm, "data/logBB_svm_morgan.pkl", compress=3)

['data/logBB_svm_morgan.pkl']

In [50]:
# predict logBB for the test set compounds
pred_svm = svm.predict(x_ts)

In [51]:
pred_svm

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [52]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_svm))
print("MCC = ", matthews_corrcoef(y_ts, pred_svm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_svm))

Accuracy =  0.7846153846153846
MCC =  0.5525513059108836
Kappa =  0.5417925478348439


In [53]:
# estimate applicability domain and calc stat
pred_prob = svm.predict_proba(x_ts)

In [54]:
da = np.amax(pred_prob, axis=1) > threshold

In [55]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred_svm[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred_svm[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred_svm[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.8571428571428571
MCC =  0.7083333333333334
Kappa =  0.7083333333333333
Coverage =  0.4307692307692308


### build the third model (GBM) compute consensus predictions from RF, and SVM models

In [57]:
# setup model building
param_grid = {"n_estimators": [100, 200, 300, 400, 500]}
gbm = GridSearchCV(GradientBoostingClassifier(subsample=0.5, max_features=0.5), 
                   param_grid, n_jobs=2, cv=cv, verbose=1)

In [58]:
# run model building
gbm.fit(x_tr, y_tr)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(max_features=0.5,
                                                  subsample=0.5),
             n_jobs=2, param_grid={'n_estimators': [100, 200, 300, 400, 500]},
             verbose=1)

In [59]:
gbm.best_score_

0.7773001508295625

In [60]:
gbm.best_params_

{'n_estimators': 200}

In [61]:
pred_gbm = gbm.predict(x_ts)

In [62]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_gbm))
print("MCC = ", matthews_corrcoef(y_ts, pred_gbm))
print("Kappa = ", cohen_kappa_score(y_ts, pred_gbm))

Accuracy =  0.7846153846153846
MCC =  0.5528589525751528
Kappa =  0.5517241379310345


#### consensus model

In [63]:
pred_c = 1 * (((pred_rf + pred_svm + pred_gbm) / 3) >= 0.5)

In [60]:
pred_c

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [64]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred_c))
print("MCC = ", matthews_corrcoef(y_ts, pred_c))
print("Kappa = ", cohen_kappa_score(y_ts, pred_c))

Accuracy =  0.7846153846153846
MCC =  0.5528589525751528
Kappa =  0.5517241379310345


### Add to Morgan fingerprints some other descriptors and look at the model performance

In [65]:
# calc some descriptors
descr = []
for m in mols:
    descr.append([Descriptors.MolLogP(m),
                  Descriptors.TPSA(m),
                  Descriptors.NHOHCount(m),
                  Descriptors.NOCount(m),
                  Descriptors.NumHAcceptors(m),
                  Descriptors.NumHDonors(m),
                  Descriptors.NumRotatableBonds(m),
                  Descriptors.NumHeteroatoms(m),
                  Descriptors.FractionCSP3(m)])
descr = np.asarray(descr)

In [66]:
descr.shape

(321, 9)

In [67]:
# add them to morgan fingerprints
x = np.concatenate((x, descr), axis=1)

In [68]:
x.shape

(321, 2057)

In [69]:
# randomly select 20% of compounds as test set
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed)

In [70]:
scale = StandardScaler().fit(x_tr)
x_tr = scale.transform(x_tr)

In [71]:
# create grid search dictionary
param_grid = {"max_features": [x_tr.shape[1] // 10, x_tr.shape[1] // 7, x_tr.shape[1] // 5, x_tr.shape[1] // 3], 
              "n_estimators": [100, 250, 500]}

In [72]:
# setup model building
m = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=2, cv=cv, verbose=1)

In [73]:
# run model building
m.fit(x_tr, y_tr)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(), n_jobs=2,
             param_grid={'max_features': [205, 293, 411, 685],
                         'n_estimators': [100, 250, 500]},
             verbose=1)

In [74]:
m.best_score_

0.8202865761689291

In [75]:
# scale descriptors of the test set compounds
x_ts = scale.transform(x_ts)

In [76]:
# predict logBB for the test set compounds
pred = m.predict(x_ts)

In [77]:
pred

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1])

In [78]:
# calc statistics
print("Accuracy = ", accuracy_score(y_ts, pred))
print("MCC = ", matthews_corrcoef(y_ts, pred))
print("Kappa = ", cohen_kappa_score(y_ts, pred))

Accuracy =  0.8615384615384616
MCC =  0.7135886373837685
Kappa =  0.7102526002971767


In [79]:
# estimate applicability domain and calc stat
pred_prob = m.predict_proba(x_ts)

In [80]:
da = np.amax(pred_prob, axis=1) > threshold

In [81]:
print("Accuracy = ", accuracy_score(np.asarray(y_ts)[da], pred[da]))
print("MCC = ", matthews_corrcoef(np.asarray(y_ts)[da], pred[da]))
print("Kappa = ", cohen_kappa_score(np.asarray(y_ts)[da], pred[da]))
print("Coverage = ", sum(da) / len(da))

Accuracy =  0.9459459459459459
MCC =  0.888015443881146
Kappa =  0.8817891373801917
Coverage =  0.5692307692307692


The model has a better accuracy. Added descritors improved the model predictivity.

#### Let's try to analyse which variables are the most important in the model

In [82]:
# rebuild RF model manually using best parameters to be able to extract additional information from the model
rf = RandomForestClassifier(n_estimators=m.best_params_["n_estimators"], 
                           max_features=m.best_params_["max_features"],
                           random_state=seed)
rf.fit(x_tr, y_tr)

RandomForestClassifier(max_features=293, n_estimators=500, random_state=42)

In [83]:
imp = rf.feature_importances_

In [84]:
imp

array([0.        , 0.00211988, 0.00034081, ..., 0.01004565, 0.03644454,
       0.02356641])

In [85]:
indices = np.argsort(imp)[::-1]

print("Feature ranking:")

# print top 10 features
for i in range(10):
    print("%d. feature %d (%f)" % (i + 1, indices[i], imp[indices[i]]))

Feature ranking:
1. feature 2049 (0.118598)
2. feature 2051 (0.055542)
3. feature 2048 (0.048918)
4. feature 2055 (0.036445)
5. feature 650 (0.035974)
6. feature 2052 (0.033922)
7. feature 2056 (0.023566)
8. feature 2050 (0.019445)
9. feature 2053 (0.017630)
10. feature 807 (0.012406)


2049 - MolLogP  
2050 - TPSA(m)  
2051 - NHOHCount  
2052 - NOCount 
2053 - NumHAcceptors  
2054 - NumHDonors  
2055 - NumRotatableBonds  
2056 - NumHeteroatoms  
2057 - FractionCSP3

features with numbers 1-2048 are different Morgan fingerprints  